In [1]:
print('hello world')

hello world


In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import unicodedata
import tensorflow_datasets as tfds
import tensorboardcolab as tbc
from tqdm import tqdm_notebook



In [0]:
# tfds.list_builders()

In [0]:
# Will use civil_comments to detect only toxicity from text

ds_train, meta = tfds.load(name = 'imdb_reviews', with_info=True, split=['train', 'test'])

In [0]:
train = ds_train[0]
valid = ds_train[1]

In [0]:
for data in train.take(1):
  print(data)

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}


In [0]:
for n, (label, text) in enumerate(train):
  print(data)
  print('--------------')
  # print(data['label'])
  print(label)
  # print(data['text'])
  print(text)
  if n == 0:
    break

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}
--------------
label
text


In [0]:
# Build Tokenizer
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus((data['text'].numpy() for data in train), 
                                                                     target_vocab_size=2**15)

In [0]:
test_sentence = 'this sentence is the test sentence for the tokenizer'

tokenizer.encode(test_sentence)

[14, 10057, 8, 1, 3155, 10057, 22, 1, 330, 9097, 14561, 32508]

In [0]:
max_word_num = len(tokenizer.subwords)

print(max_word_num)

32393


In [0]:
tokenizer.vocab_size

32650

In [0]:
next(iter(train))

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}

In [0]:
BATCH_SIZE = 64

In [0]:
text_data = train.map(lambda x: x['text'])
label_data = train.map(lambda x: x['label'])

data_zip = tf.data.Dataset.zip((text_data, label_data))

In [0]:
for n, data in enumerate(data_zip):
  print(data)
  # print(label)
  if n == 0:
    break

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [0]:
def text_encode(data, label):
  tokenized_result = tokenizer.encode(data.numpy()) # requires to set bracket!!
  return np.array(tokenized_result), label

In [0]:
def tf_text_encode(data, label):
  result_text, label_ = tf.py_function(text_encode, [data, label], [tf.int64, tf.int64])
  # result_text = tf.cast(result_text, tf.float64)
  label_ = tf.cast(label_, tf.float64)
  return result_text, label_

In [0]:
# def label_encode(text, label):
#   if label >= 0.5:
#     return text, 1.
#   elif label < 0.5 :
#     return text, 0.

In [0]:
# def tf_label_encode(text, label):
#   text, result_label = tf.py_function(label_encode, [text, label], [tf.int32, tf.float64])
#   return text, result_label

In [0]:
dataset = data_zip.map(tf_text_encode)
dataset = dataset.padded_batch(32, padded_shapes=([None], []))
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
dataset

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>

In [0]:
for n, data in enumerate(dataset):
  print(data)
  # print(data[0])
  if n == 0:
    break

(<tf.Tensor: shape=(64, 924), dtype=int64, numpy=
array([[   56,    18,    41, ...,     0,     0,     0],
       [   12,    31,    85, ...,     0,     0,     0],
       [ 8352, 12617,     1, ...,     0,     0,     0],
       ...,
       [  122,  1199,     5, ...,     0,     0,     0],
       [20576,    44,    14, ...,     0,     0,     0],
       [   12,   177, 32433, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.])>)


In [0]:
valid

<DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>

In [0]:
valid_text_data = valid.map(lambda x: x['text'])
valid_label_data = valid.map(lambda x: x['label'])
data_zip = tf.data.Dataset.zip((valid_text_data, valid_label_data))
valid_dataset = data_zip.map(tf_text_encode)
valid_dataset = valid_dataset.padded_batch(32, padded_shapes=([None], []))
valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
print(valid_dataset)
print(dataset)

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>
<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>


In [0]:
for n, data in enumerate(dataset):
  print(data)
  if n == 0:
    break

(<tf.Tensor: shape=(64, 924), dtype=int64, numpy=
array([[   56,    18,    41, ...,     0,     0,     0],
       [   12,    31,    85, ...,     0,     0,     0],
       [ 8352, 12617,     1, ...,     0,     0,     0],
       ...,
       [  122,  1199,     5, ...,     0,     0,     0],
       [20576,    44,    14, ...,     0,     0,     0],
       [   12,   177, 32433, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.])>)


In [0]:
tf.random.set_seed(1228)

## Hirarchical Attention Layer

In [0]:
class HAN_Attention(tf.keras.layers.Layer):
  '''
  Take GRU: Sequences
  Output: Context Vector & Attention Matrix
  '''
  def __init__(self, units):
    super(HAN_Attention, self).__init__()
    self.units = units

    self.S = tf.keras.layers.Dense(units)
    self.U = tf.keras.layers.Dense(1, use_bias=False)
  
  def call(self, inputs): # input = B, T, U
    U_it = tf.nn.tanh(self.S(inputs))
    A_it = self.U(U_it)
    A_it = tf.nn.softmax(A_it, axis=1)  # B, T, 1

    V = inputs * A_it
    V = tf.reduce_sum(V, axis=1)
    return V

In [0]:
attention = HAN_Attention(128)

In [0]:
attention(tf.random.normal(shape=(1, 100,128)))

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.10158738, -0.09020579, -0.16854474, -0.21384944,  0.08745283,
         0.13345653, -0.10566922, -0.20149784,  0.05222992,  0.0767891 ,
        -0.0387913 , -0.23818506, -0.14838322,  0.00558304, -0.10479707,
         0.23523614,  0.0344021 , -0.14978659, -0.03695203,  0.10442456,
        -0.16012375, -0.0458557 , -0.212784  ,  0.00690812, -0.07473513,
         0.02402451, -0.24763177,  0.12444421,  0.05835668,  0.14326297,
         0.02509566,  0.15165958,  0.09170369,  0.1786546 ,  0.08701997,
         0.05246879, -0.02575276,  0.08496442,  0.06299901,  0.01866038,
        -0.07285462,  0.03339475,  0.01968693, -0.00250611, -0.11362679,
         0.03919471,  0.12804784,  0.12449674,  0.05881004, -0.01578162,
         0.06516027, -0.24417013, -0.10587377,  0.22663698,  0.10968988,
        -0.15267184,  0.02636713,  0.11128744,  0.19007134,  0.05004203,
        -0.01263488,  0.15219584, -0.08893486, -0.050078  ,  0.00613859,
 

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.GRU(256, return_sequences=True)(x)
  x = HAN_Attention(256)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_23 (Embedding)     (None, None, 300)         9795000   
_________________________________________________________________
gru_23 (GRU)                 (None, None, 256)         428544    
_________________________________________________________________
han__attention_33 (HAN_Atten (None, 256)               66048     
_________________________________________________________________
dense_82 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 129       
Total params: 10,322,617
Trainable params: 10,322,617
Non-trainable params: 0
_______________________________________________

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_1 = model.fit(dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100
391/391 [==============================] - 138s 354ms/step - loss: 0.3459 - accuracy: 0.8430 - val_loss: 0.3202 - val_accuracy: 0.8647
Epoch 2/100
391/391 [==============================] - 138s 352ms/step - loss: 0.1195 - accuracy: 0.9572 - val_loss: 0.3651 - val_accuracy: 0.8827
Epoch 3/100
391/391 [==============================] - 137s 350ms/step - loss: 0.0752 - accuracy: 0.9729 - val_loss: 0.4411 - val_accuracy: 0.8814
Epoch 4/100
391/391 [==============================] - 136s 347ms/step - loss: 0.0271 - accuracy: 0.9907 - val_loss: 0.5313 - val_accuracy: 0.8802
Epoch 5/100
391/391 [==============================] - 136s 348ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.8378 - val_accuracy: 0.8580
Epoch 6/100
391/391 [==============================] - 136s 347ms/step - loss: 0.0033 - accuracy: 0.9991 - val_loss: 0.7596 - val_accuracy: 0.8723
Epoch 7/100
391/391 [==============================] - 136s 347ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 1

KeyboardInterrupt: ignored

## Attention with Bidirection

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.SpatialDropout1D(0.3)(x)
  x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)
  x = HAN_Attention(256)(x)
  x = tf.keras.layers.Dense(128)(x)
  x = tf.keras.layers.LeakyReLU()(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 300)         9795000   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, None, 300)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 256)         330240    
_________________________________________________________________
han__attention_3 (HAN_Attent (None, 256)               66048     
_________________________________________________________________
dense_58 (Dense)             (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0   

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_2 = model.fit(dataset, epochs=50, validation_data=valid_dataset)

Epoch 1/50
    677/Unknown - 4699s 7s/step - loss: 0.3542 - accuracy: 0.8365

## Re..


In [0]:
from tensorflow.keras import backend as K

In [0]:
class Attention_layer(tf.keras.layers.Layer):
  def __init__(self, units):
    super(Attention_layer, self).__init__()
    self.units = units

    self.w = tf.keras.layers.Dense(units)
    self.u = tf.keras.layers.Dense(1)

  def call(self, inputs):
    uit = self.w(inputs)
    uit = tf.nn.tanh(uit)

    ait_base = self.u(uit)
    ait = tf.exp(tf.squeeze(self.u(uit), -1))
    ait /= tf.reduce_sum(ait, axis=1, keepdims=True)
    ait = tf.expand_dims(ait, -1)
    print('keras version ait', ait)


    ait2 = tf.nn.softmax(ait_base, axis=1)
    print('softmax function', ait2)

    si = ait2 * inputs  # * mark and @ mark is the same
    si = tf.reduce_sum(si, axis=1)
    
    # si2 = ait @ inputs
    # si2 = tf.reduce_sum(si2)
    return si

In [0]:
test_input = tf.random.normal(shape=(1, 5, 10))

attention = Attention_layer(10)

attention(test_input)

keras version ait tf.Tensor(
[[[0.04669954]
  [0.05252406]
  [0.40741324]
  [0.32868347]
  [0.16467969]]], shape=(1, 5, 1), dtype=float32)
softmax function tf.Tensor(
[[[0.04669954]
  [0.05252407]
  [0.40741324]
  [0.32868347]
  [0.1646797 ]]], shape=(1, 5, 1), dtype=float32)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.19133395,  0.90555465,  0.8815074 , -0.08758724, -0.24763313,
        -0.8968195 , -0.04773174, -0.23755713,  0.7508702 , -0.18253191]],
      dtype=float32)>

In [0]:
test_constant = tf.constant([[[1.,2.,3.], [4.,5.,6.]],
                             [[10., 11., 12.], [20., 21., 22]]])

In [0]:
test_constant

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[10., 11., 12.],
        [20., 21., 22.]]], dtype=float32)>

In [0]:
tf.reduce_sum(test_constant)

<tf.Tensor: shape=(), dtype=float32, numpy=117.0>

In [0]:
tf.reduce_sum(test_constant, axis=0)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 13., 15.],
       [24., 26., 28.]], dtype=float32)>

In [0]:
tf.reduce_sum(test_constant, axis=1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 5.,  7.,  9.],
       [30., 32., 34.]], dtype=float32)>

In [0]:
tf.reduce_sum(test_constant, axis=2)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 6., 15.],
       [33., 63.]], dtype=float32)>

## Comparison From other github sources

In [0]:
# Check Dense(10) weights
# Check Dense(1) weights

# Check Build method 

In [0]:
test_input = tf.random.normal(shape=(1, 10))

In [0]:
test_input_holder = tf.keras.layers.Input(shape=(None,20,10))
check_1 = tf.keras.layers.Dense(10, use_bias=False)(test_input_holder)

In [0]:
check_1

<tf.Tensor 'dense_6/Identity:0' shape=(None, None, 20, 10) dtype=float32>

In [0]:
model = tf.keras.models.Model(test_input_holder, check_1)

In [0]:
model.layers[1].weights

# Input - shape = (B, U)
# the weights = (input_shape, units) # if the input_shape is dim2

# Input - shape = (B, t, U)
# the weight = (input_shape[2], units)

# Bias : (U,)

[<tf.Variable 'dense_6/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.39734095, -0.30013412, -0.34879762, -0.44497746, -0.20599052,
         -0.15094578, -0.37742782,  0.25683457,  0.17134011, -0.2417103 ],
        [-0.16097698, -0.18883136, -0.13476276,  0.22899449, -0.05796689,
          0.11870927,  0.03996646,  0.18723142, -0.03622872, -0.15857118],
        [-0.5200933 ,  0.03484213, -0.10524586, -0.541772  ,  0.17962223,
          0.41390693, -0.45359883, -0.03697866, -0.5390602 , -0.01256704],
        [-0.11791846, -0.5306347 ,  0.20147651, -0.08126822, -0.2545558 ,
          0.502061  , -0.545898  ,  0.42779398, -0.13202578, -0.35113576],
        [ 0.5238167 , -0.11403191,  0.27053708, -0.4075817 ,  0.25372946,
         -0.5148867 , -0.5240577 , -0.36272082,  0.38691455, -0.14195004],
        [ 0.49883807,  0.36792862, -0.5084392 ,  0.28085828, -0.07757914,
          0.38492495, -0.21936268,  0.14643621,  0.51880634, -0.49611634],
        [-0.1150603 , -0.0474785 ,  

In [0]:
test_input_holder = tf.keras.layers.Input(shape=(None,20))
check_1 = tf.keras.layers.Dense(10)(test_input_holder)
model = tf.keras.models.Model(test_input_holder, check_1)

In [0]:
model.layers[1].weights

[<tf.Variable 'dense_9/kernel:0' shape=(20, 10) dtype=float32, numpy=
 array([[-0.23868589, -0.22021157, -0.43113896,  0.35091853,  0.09421682,
         -0.08422348,  0.3058331 , -0.09196609,  0.15912056, -0.40685618],
        [ 0.3165061 ,  0.3745914 ,  0.1577105 ,  0.10152113,  0.11089152,
          0.15097558,  0.14456451, -0.40147093,  0.28108406,  0.16155142],
        [ 0.2410686 , -0.24997236, -0.08986327,  0.42702818,  0.2604798 ,
         -0.23813997, -0.02206552,  0.07003409,  0.25749588,  0.22565812],
        [-0.25441656,  0.30738664,  0.44680947,  0.05290014,  0.1296224 ,
         -0.02711129, -0.10263404,  0.23800576,  0.16010708, -0.01825115],
        [-0.16431862, -0.02844441, -0.20703144, -0.03822365,  0.17901927,
         -0.15476382,  0.4069773 , -0.1224978 , -0.42503962, -0.37858343],
        [-0.2467987 ,  0.4371258 , -0.26136503, -0.23020022, -0.29986483,
          0.40231103,  0.42372113,  0.30585366,  0.10941458,  0.0039258 ],
        [ 0.30979002, -0.3879111 , -

In [0]:
test_input_holder = tf.keras.layers.Input(shape=(None,20))
check_1 = tf.keras.layers.Dense(1)(test_input_holder)
model = tf.keras.models.Model(test_input_holder, check_1)
model.layers[1].weights

[<tf.Variable 'dense_11/kernel:0' shape=(20, 1) dtype=float32, numpy=
 array([[-0.40410066],
        [-0.11591452],
        [-0.13106042],
        [ 0.51371187],
        [ 0.02033496],
        [-0.06176868],
        [-0.3037858 ],
        [-0.03395081],
        [-0.31504694],
        [-0.31896013],
        [ 0.45716238],
        [ 0.12727952],
        [-0.13587636],
        [ 0.45334393],
        [-0.44746906],
        [ 0.53445214],
        [ 0.2794246 ],
        [ 0.13106489],
        [-0.427267  ],
        [-0.2888232 ]], dtype=float32)>,
 <tf.Variable 'dense_11/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [0]:
class Linear(tf.keras.layers.Layer):
  def __init__(self, units):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                             initializer='random_normal',
                             trainable= True)
    self.b = self.add_weight(shape=(self.units, ),
                             initializer='zeros',
                             trainable=True)
  
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [0]:
linear = Linear(10)

In [0]:
linear(test_input_holder)

<tf.Tensor 'linear/Identity:0' shape=(None, None, 20, 10) dtype=float32>